In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

In [117]:
#Load CSV
# df = pd.read_csv("../mergedata_%change.csv")
df = pd.read_csv("../PRE_POST_VOL")
df.head()

,Unnamed: 0,Pre_Tweet_Vol,Post_Tweet_Vol,Vol Change,Tweet,Tweet Time
0,551,-0.010892,0.001412,0.012304,Just had a very good tele-conference with Nat...,20200316 1332
1,915,-0.008380,0.003885,0.012264,The Federal Reserve is cutting but must furthe...,20200303 1051
2,465,0.008989,0.018959,0.009970,Thank you @HerschelWalker! https://t.co/qBXOa4...,20200318 1550
3,7269,-0.007727,0.001513,0.009240,Today we honor all of our Country’s Purple Hea...,20190807 0946
4,1031,-0.005200,0.003462,0.008661,“You go around Pennsylvania and you see Trump ...,20200228 1030


In [126]:
#Bin SPX % Change into +1,0,-1 and establish threshold
market_movements = []
pos_threshold = 0.0003
neg_threshold = pos_threshold * -1

for index, row in df.iterrows():
    if row['Vol Change'] >= pos_threshold:
        market_movement = 1
        market_movements.append(market_movement)
    elif row['Vol Change'] <= neg_threshold:
        market_movement = -1
        market_movements.append(market_movement)
    else:
        market_movement = 0
        market_movements.append(market_movement)

#Add new list as column to df
df['Market Movement'] = market_movements
df

,Unnamed: 0,Pre_Tweet_Vol,Post_Tweet_Vol,Vol Change,Tweet,Tweet Time,Market Movement
0,551,-0.010892,0.001412,0.012304,Just had a very good tele-conference with Nat...,20200316 1332,1
1,915,-0.008380,0.003885,0.012264,The Federal Reserve is cutting but must furthe...,20200303 1051,1
2,465,0.008989,0.018959,0.009970,Thank you @HerschelWalker! https://t.co/qBXOa4...,20200318 1550,1
3,7269,-0.007727,0.001513,0.009240,Today we honor all of our Country’s Purple Hea...,20190807 0946,1
4,1031,-0.005200,0.003462,0.008661,“You go around Pennsylvania and you see Trump ...,20200228 1030,1
...,...,...,...,...,...,...,...
2304,717,0.008702,-0.000290,-0.008992,Thank you @SenateGOP! https://t.co/XysOvXbTMF,20200310 1144,-1
2305,839,0.006164,-0.003560,-0.009724,Gallup just gave us the highest rating ever fo...,20200305 1534,-1
2306,10724,0.007377,-0.002386,-0.009762,The reason the DACA for Wall deal didn’t get d...,20181227 1444,-1
2307,649,0.027447,0.003998,-0.023449,KEEPING THE HOME SAFE ENCOURAGE YOUR FAMILY ME...,20200312 1301,-1


In [127]:
#State target output, name the %change bins
target = df["Market Movement"]
target_names = ["Positive", "No Effect", "Negative"]

In [128]:
#Remove Output column to avoid data leakage
data = df.drop("Market Movement", axis=1)
feature_names = data.columns
data.head()

,Unnamed: 0,Pre_Tweet_Vol,Post_Tweet_Vol,Vol Change,Tweet,Tweet Time
0,551,-0.010892,0.001412,0.012304,Just had a very good tele-conference with Nat...,20200316 1332
1,915,-0.008380,0.003885,0.012264,The Federal Reserve is cutting but must furthe...,20200303 1051
2,465,0.008989,0.018959,0.009970,Thank you @HerschelWalker! https://t.co/qBXOa4...,20200318 1550
3,7269,-0.007727,0.001513,0.009240,Today we honor all of our Country’s Purple Hea...,20190807 0946
4,1031,-0.005200,0.003462,0.008661,“You go around Pennsylvania and you see Trump ...,20200228 1030


In [129]:
# tokenize, then remove stopwords, ngrams, sentiment analysis feature selection
#either truncate or append to vector
data.shape

(2309, 6)

## Count Vectorizer

In [134]:
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer

corpus = data['Tweet']

vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1, 3), stop_words = 'english')
vector = vectorizer2.fit_transform(corpus)
vector

<2309x59192 sparse matrix of type '<class 'numpy.int64'>'
	with 91492 stored elements in Compressed Sparse Row format>

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42575636)

print(X_train.shape)
print(X_test.shape)

(1731, 59192)
(578, 59192)


In [136]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [137]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

Test Acc: 0.3702
              precision    recall  f1-score   support

    Positive       0.33      0.45      0.38       187
   No Effect       0.40      0.24      0.30       178
    Negative       0.40      0.41      0.40       213

    accuracy                           0.37       578
   macro avg       0.38      0.37      0.36       578
weighted avg       0.38      0.37      0.36       578



## Hashing Vectorizer

In [44]:
from sklearn.feature_extraction.text import HashingVectorizer

# list of text documents
text = data["Tweet"]
# create the transform
vectorizer = HashingVectorizer(n_features=2000, ngram_range=(1, 3))
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(2955, 2000)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.08006408  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.10540926  0.          0.         ...  0.          0.
   0.        ]]


In [45]:
vector

<2955x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 226631 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42)

In [47]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [48]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

Test Acc: 0.4073


In [49]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    Positive       0.41      0.48      0.44       310
   No Effect       0.00      0.00      0.00       102
    Negative       0.41      0.47      0.44       327

    accuracy                           0.41       739
   macro avg       0.27      0.32      0.29       739
weighted avg       0.35      0.41      0.38       739



C:\Users\Natha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
target = data["RTs"]
target_names = ["Positive", "No Effect", "Negative"]

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector.toarray(), target, random_state=42)

In [49]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [50]:
print('Test Acc: %.4f' % model.score(X_test, y_test))

Test Acc: 0.0200
